---
#STEP 0: Getting everything ready


First we need to create the right folders

In [1]:
import os
import cv2
general_videos_folder = "./videos/"
inputs_subfolder = "inputs/"
audio_subfolder = "audios/"
frames_subfolder = "frames/"
processed_frames_subfolder = "processed_frames/"
no_sound_videos_subfolder = "no_sound_videos/"
final_videos_subfolder = "final_videos/"
os.makedirs(general_videos_folder, exist_ok = True)
os.makedirs(f"{general_videos_folder}{inputs_subfolder}", exist_ok = True)
os.makedirs(f"{general_videos_folder}{audio_subfolder}", exist_ok = True)
os.makedirs(f"{general_videos_folder}{frames_subfolder}", exist_ok = True)
os.makedirs(f"{general_videos_folder}{processed_frames_subfolder}", exist_ok = True)
os.makedirs(f"{general_videos_folder}{no_sound_videos_subfolder}", exist_ok = True)
os.makedirs(f"{general_videos_folder}{final_videos_subfolder}", exist_ok = True)

Now please download a video (I will be using the video from [this link](https://drive.google.com/file/d/1AT1SlrUrzbEdQzr301CnlkCZ8DNC4ULf/view?usp=sharing)), and put it in the "videos/inputs/" folder.

Give it a name (for me it was  **`the_office_auditions.mp4`** )

Now run the following box with the name you gave it (do not include the .mp4 at the end)



In [2]:
filename = "news_base"

---
#STEP 1: Break a video into multiple *frames*

For this we use the cv2 library. This code is based on the [following link](https://www.geeksforgeeks.org/python-program-extract-frames-using-opencv/)

In [4]:
def video_to_frame(filename):
  os.makedirs(f"{general_videos_folder}{frames_subfolder}{filename}/", exist_ok = True)
  get_frame_path = lambda part: f"{general_videos_folder}{frames_subfolder}{filename}/frame-{part}.jpg" 
  vid_obj = cv2.VideoCapture(f"{general_videos_folder}{inputs_subfolder}{filename}.mp4")
  count = 0
  print(f"Total # of frames according to code {int(vid_obj.get(cv2.CAP_PROP_FRAME_COUNT))}")
  fps = vid_obj.get(cv2.CAP_PROP_FPS)
  print(f"Frames per second = {fps}")
  print("Now reading the frames... [This might take a while]")
  while True: 
    success, image = vid_obj.read() 
    if not success:
      break
    # Saves the frames with frame-count 
    cv2.imwrite(get_frame_path(count), image) 
    count += 1
  print(f"Total # frames accroding to loop = {count}")
  print(f"Succesfully wrote all the frames in the folder {general_videos_folder}{frames_subfolder}{filename}/")
  return fps

Apply function to our video


In [5]:
video_fps = video_to_frame(filename) #this function return the fps of such video
print(f"FPS returned from function: {video_fps}")

Total # of frames according to code 150
Frames per second = 29.97002997002997
Now reading the frames... [This might take a while]
Total # frames accroding to loop = 150
Succesfully wrote all the frames in the folder ./videos/frames/news_base/
FPS returned from function: 29.97002997002997


Run the following **only** if you want to zip the frames folder for this video so that you can download them. 

In [0]:
os.environ['FRAMES_FOLDER'] =  f"{general_videos_folder}{frames_subfolder}{filename}"
os.environ['ZIP_LOCATION'] = "."  #Will save it on the frames/ folder TODO: does this only add the frames/ folder?
!zip -r $FRAMES_FOLDER $ZIP_LOCATION

Streaming output truncated to the last 5000 lines.
  adding: videos/processed_frames/the_office_auditions/frame-7414.jpg (deflated 2%)
  adding: videos/processed_frames/the_office_auditions/frame-656.jpg (deflated 2%)
  adding: videos/processed_frames/the_office_auditions/frame-315.jpg (deflated 1%)
  adding: videos/processed_frames/the_office_auditions/frame-1081.jpg (deflated 3%)
  adding: videos/processed_frames/the_office_auditions/frame-1390.jpg (deflated 1%)
  adding: videos/processed_frames/the_office_auditions/frame-2967.jpg (deflated 2%)
  adding: videos/processed_frames/the_office_auditions/frame-5738.jpg (deflated 2%)
  adding: videos/processed_frames/the_office_auditions/frame-5118.jpg (deflated 2%)
  adding: videos/processed_frames/the_office_auditions/frame-4250.jpg (deflated 1%)
  adding: videos/processed_frames/the_office_auditions/frame-6314.jpg (deflated 2%)
  adding: videos/processed_frames/the_office_auditions/frame-705.jpg (deflated 2%)
  adding: videos/processed_f

---
#STEP 2: Updating every frame 

In [0]:
#For now, we will just copy the same images
os.environ['FRAME_FOLDER'] = f"{general_videos_folder}{frames_subfolder}{filename}"
os.environ['PROCESSED_FRAME_FOLDER'] = f"{general_videos_folder}{processed_frames_subfolder}{filename}"
!cp -r $FRAME_FOLDER $PROCESSED_FRAME_FOLDER

---
#STEP 3: Outputting a video from several frames


Helper method that will help us make sure that names like 'frame-2' goes before 'frame-10'

In [0]:
def extract_number_from_image(filename, termination = ".jpg"):
    for i in range(len(filename)): #going through every character
        char = filename[i]
        if char.isdecimal():       #we found the beginning of the number we care about!
            last_decimal_index = len(filename) - len(termination) -1
            num_extracted = filename[i:last_decimal_index + 1]
            if num_extracted.isdecimal():
                return int(num_extracted)
            else:
                raise NameError("The filename {} should be of the form [PREFIX][NUMBER][TERMINATION]".format(filename))
    raise NameError("The filename {} didnt have any decimal digits".format(filename))

We do this in two parts

1.   First, we sort the images in the necessary folder. We use the `extract_number_from_image` function above to make sure that `frame-2.jpg` goes before `frame-10.jpg` 
2.   For every image of the frame, we get its raw value by calling `cv2.imread()` and we add each of them to a `VideoWriter` object previously created.
> We create this object by using the size of the first image. 





In [0]:
video_fps = 30 #just for now, dont forget to delete

In [0]:
def frames_to_video(filename):
  in_folder = f"{general_videos_folder}{processed_frames_subfolder}"
  all_elements = os.listdir(f"{in_folder}{filename}/")
  all_images = list(filter(lambda name: name.endswith(".jpg"), all_elements)) #only keeping the images
  all_images.sort(key = extract_number_from_image) #to order them by number, and not lexicographically
  print("Going through all frames")
  is_first = True
  for image in all_images:
    image_path = f"{in_folder}{filename}/{image}"
    print(f"Image path: {image_path}")
    frame = cv2.imread(image_path)
    height, width, layers = frame.shape
    size = (width,height)
    if is_first:
      out_path = f"{general_videos_folder}{no_sound_videos_subfolder}frames_combined_{filename}.avi" #TODO: find fourcc of MP4 files
      print(f"Creating a new video file at {out_path} with fps ={video_fps} and size {size}")
      fourcc = cv2.VideoWriter_fourcc(*'DIVX')
      out = cv2.VideoWriter(out_path,fourcc, video_fps, size) #creating a video to store the frames
      is_first = False
    out.write(frame)
  out.release()
  cv2.destroyAllWindows() 
  print("Finished converting to video!")

In [0]:
def frames_to_video_helper():
  in_folder = f"{general_videos_folder}{processed_frames_subfolder}"
  all_elements = os.listdir(f"{in_folder}{filename}/")
  all_images = list(filter(lambda name: name.endswith(".jpg"), all_elements)) #only keeping the images
  all_images.sort(key = extract_number_from_image) #to order them by number, and not lexicographically
  print("Going through all frames")
  is_first = True
  for image in all_images:
    image_path = f"{in_folder}{filename}/{image}"
    print(f"Image path: {image_path}")
    frame = cv2.imread(image_path)
    height, width, layers = frame.shape
    size = (width,height)
    if is_first:
      out_path = f"{general_videos_folder}{no_sound_videos_subfolder}frames_combined_{filename}.avi" #TODO: find fourcc of MP4 files
      print(f"Creating a new video file at {out_path} with fps ={video_fps} and size {size}")
      fourcc = cv2.VideoWriter_fourcc(*'DIVX')
      out = cv2.VideoWriter(out_path,fourcc, video_fps, size) #creating a video to store the frames
      is_first = False
    out.write(frame)
  out.release()
  cv2.destroyAllWindows() 
  print("Finished converting to video!")

Now lets convert the processed frames we got into a sweet video (but with no sound)


In [0]:
frames_to_video(filename)

Going through all frames
Image path: ./videos/processed_frames/the_office_auditions/frame242.jpg
Creating a new video file at ./videos/no_sound_videos/frames_combined_the_office_auditions.avi with fps =30 and size (496, 360)
Image path: ./videos/processed_frames/the_office_auditions/frame243.jpg
Image path: ./videos/processed_frames/the_office_auditions/frame244.jpg
Image path: ./videos/processed_frames/the_office_auditions/frame245.jpg
Image path: ./videos/processed_frames/the_office_auditions/frame246.jpg
Image path: ./videos/processed_frames/the_office_auditions/frame247.jpg
Image path: ./videos/processed_frames/the_office_auditions/frame248.jpg
Image path: ./videos/processed_frames/the_office_auditions/frame249.jpg
Image path: ./videos/processed_frames/the_office_auditions/frame250.jpg
Image path: ./videos/processed_frames/the_office_auditions/frame251.jpg
Image path: ./videos/processed_frames/the_office_auditions/frame252.jpg
Image path: ./videos/processed_frames/the_office_auditi

---

#STEP 4: Extract the audio from original video


Now, even though we just created a video out of our processed frames, we lost the sound in the process, so we can try to extract the audio from the original video as follows:



In [0]:
from moviepy.editor import AudioFileClip
def extract_audio(filename):
  audio_path = f"{general_videos_folder}{inputs_subfolder}{filename}.mp4"
  audio = AudioFileClip(audio_path)
  audio.write_audiofile(f"{general_videos_folder}{audio_subfolder}audio_from_{filename}.mp3")

Now lets extract the audio of the video we created first

In [0]:
extract_audio(filename)

[MoviePy] Writing audio in ./videos/audios/audio_from_the_office_auditions.mp3


100%|██████████| 7520/7520 [00:08<00:00, 917.09it/s]

[MoviePy] Done.


---

#STEP 5: Add the audio to our processed video


We are so close! Now we only need a way to combine the video we created in `Step #3` and the original audio we extracted in `Step #4`

In [0]:
from moviepy.editor import VideoFileClip
def add_audio_to_processed_video(filename):
  video_no_sound_path = f"{general_videos_folder}{no_sound_videos_subfolder}frames_combined_{filename}.avi"
  video_no_sound = VideoFileClip(video_no_sound_path)

  video_final_path = f"{general_videos_folder}{final_videos_subfolder}final_{filename}.mp4"
  audio_path = f"{general_videos_folder}{audio_subfolder}audio_from_{filename}.mp3"
  video_no_sound.write_videofile(video_final_path, audio = audio_path)

And now ... **drumrolls**... the final step. 

In [0]:
add_audio_to_processed_video(filename)

[MoviePy] >>>> Building video ./videos/final_videos/final_the_office_auditions.mp4
[MoviePy] Writing video ./videos/final_videos/final_the_office_auditions.mp4


100%|██████████| 10228/10228 [01:40<00:00, 102.04it/s]


[MoviePy] Done.
[MoviePy] >>>> Video ready: ./videos/final_videos/final_the_office_auditions.mp4 



Now you can see the video in `videos/final_videos/`. Please download it to make sure that:


1.   The audio is in sync with the video (this is why we needed to get the fps of the original video in `Step #`1)
2.   The frames are in the order they appeared initially
3.   The video feels *smooth* , so that it's not noticeable that this was completely done in Python!
4. For now, since in `Step #2` we only copied the frames as they were, this output video should be **identical** to the original.

**The End.**

---
